<div class="div_image pull-right">
    <div class = "image image_topic pull-right">
        <img src = "https://i.imgur.com/EGtMXKh.jpg?1">
    </div>
</div>

# <b>Capstone Project: Predicting Dota 2 Match Wins using Machine Learning and Recommender System (Recommender System)</b>


# Import Libraries

In [8]:
import pandas as pd
import tensorflow as tf
import numpy as np
import tqdm

import json

import warnings
warnings.filterwarnings('ignore')


In [9]:
# Import heroes data from json file

with open('heroes.json', 'r') as fp:
    heroes = json.load(fp)
    hero_ids = [hero['id'] for hero in heroes]

def get_hero_human_readable(hero_id):
    for hero in heroes:
        if hero['id'] == hero_id:
            return hero['localized_name']
    return 'Unknown hero: %d' % hero_id


In [15]:
#Store Variables that may change
FINAL_HERO_ID = hero_ids[-1]
NUM_FEATURES = FINAL_HERO_ID * 2
ENV_PATH = 'model.h5'
radiant_team = [45, 39, 87, 66]
dire_team = [4, 5, 13,99, 75]

missing_ids = []
for num in range(0, FINAL_HERO_ID):
    if num not in hero_ids:
        missing_ids.append(num)


In [11]:
class NNPredictor:
    '''Load Model from the environment path for us to train'''
    def __init__(self, env_path = ENV_PATH):
        self.model = tf.keras.models.load_model(env_path)

    def transform(self, my_team, their_team):
        '''Transform our inputs into the tensorflow input array of shape (1, 246). Slice list of IDs from heroes list '''
        X = np.zeros((1, (NUM_FEATURES+1)))
        '''Slice hero_id chosen onto the array, add number of heroes to hero_id for dire team'''
        for hero_id in my_team:
            X[0][(hero_id)] = 1
        for hero_id in their_team:
            X[0][(hero_id + FINAL_HERO_ID)] = 1
        
        missing_ids = []
        for num in range(0, FINAL_HERO_ID):
            if num not in hero_ids:
                 missing_ids.append(num)
        
        dire_ids = []
        for id in missing_ids:
            if id > 0:
                dire_id = id + FINAL_HERO_ID
                dire_ids.append(dire_id)

        missing_ids = missing_ids + dire_ids

        X = np.delete(X, missing_ids, axis=1)
        return X

    def recommend(self, my_team, their_team, hero_candidates):
        team_possibilities = [(candidate, my_team + [candidate]) for candidate in hero_candidates]

        prob_candidate_pairs = []
        for candidate, team in team_possibilities:
            query = self.transform(team, their_team)
            prob = self.score(query)
            prob_candidate_pairs.append((prob, candidate))
        prob_candidate_pairs = sorted(prob_candidate_pairs, reverse = True)[0:5]
        return prob_candidate_pairs

    def score(self,query):
        rad_prob_1 = self.model.predict(query, verbose = False)[0][0]
        return rad_prob_1

    def predict(self, dream_team, their_team):
        '''Returns the probability of the dream_team winning against their_team.'''
        dream_team_query = self.transform(dream_team, their_team)
        return self.score(dream_team_query)


In [12]:
def main():
    my_team = radiant_team
    their_team = dire_team
    print (f'My Team: {[get_hero_human_readable(hero_id) for hero_id in my_team]}')
    print (f'Their Team:{[get_hero_human_readable(hero_id) for hero_id in their_team]}')
    print ('Recommend:')

    engine = Engine(NNPredictor())
    recommendations = engine.recommend(my_team, their_team)
    print(f'{[(prob, get_hero_human_readable(hero))for prob, hero in recommendations]}')

In [13]:
class Engine:

    def __init__(self, algorithm):
        self.algorithm = algorithm

    def get_candidates(self, my_team, their_team):
        '''Return a list of hero_IDs to consider for recommending'''
        ids = [i for i in hero_ids if i not in my_team and i not in their_team and i not in missing_ids]
        return ids

    def recommend(self, my_team, their_team, human_readable = False):
        '''Return a list of (hero, probability of winning with hero_added) recommend to complete my_team'''

        assert len(my_team) < 5, "Your team is full! Please remove a hero from your team."
        assert len(their_team) <= 5

        hero_candidates = self.get_candidates(my_team, their_team)
        return self.algorithm.recommend(my_team, their_team, hero_candidates)

    def predict(self, dream_team, their_team):
        '''Return the probability of the dream_team winning against their team'''
        return self.algorithm.predict(dream_team, their_team)



In [16]:
if __name__ == '__main__':
    main()

My Team: ['Pugna', 'Queen of Pain', 'Disruptor', 'Chen']
Their Team:['Bloodseeker', 'Crystal Maiden', 'Puck', 'Bristleback', 'Silencer']
Recommend:
[(0.6018949, 'Visage'), (0.57422984, 'Bane'), (0.5728478, 'Winter Wyvern'), (0.56636196, 'Meepo'), (0.56110424, 'Broodmother')]
